In [ ]:
import numpy as np
import pandas as pd

import nltk


In [ ]:
# !git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

In [ ]:
df_train = pd.read_csv("DrugReviewAnalysis/drugsComTrain_raw.tsv", sep="\t")
df_test = pd.read_csv("DrugReviewAnalysis/drugsComTest_raw.tsv", sep="\t")

# df_train = pd.read_csv("drugsComTrain_raw.tsv", sep="\t")
# df_test = pd.read_csv("drugsComTest_raw.tsv", sep="\t")

#### data exploration

In [ ]:
df_train = df_train[df_train["condition"].str.contains("users found this comment helpful.")==False]
df_test = df_test[df_test["condition"].str.contains("users found this comment helpful.")==False]

In [ ]:
len(set(df_train["condition"]) & set(df_test["condition"])), len(set(df_test["condition"]))

(639, 664)

In [ ]:
len(set(df_train["drugName"]) & set(df_test["drugName"])), len(set(df_test["drugName"]))

(2387, 2629)

In [ ]:
df_train["timestamp"] = df_train["date"].apply(lambda x: pd.Timestamp(x))
df_test["timestamp"] = df_test["date"].apply(lambda x: pd.Timestamp(x))

In [ ]:
df_train = df_train.drop(columns="Unnamed: 0")
df_test = df_test.drop(columns="Unnamed: 0")

In [ ]:
df_train.head()

,drugName,condition,review,rating,date,usefulCount,timestamp
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,2012-05-20
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,2010-04-27
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,2009-12-14
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,2015-11-03
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,2016-11-27


#### preprocess(text) function implements tokenization, removing stopwards, lemmatization

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

In [ ]:
def preprocess(text):
    tokens = word_tokenize(text.lower())

    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = " ".join([re.sub('[^A-Za-z0-9]+','', x) for x in lemmatized_tokens])
    processed_text = re.sub(' +', ' ', processed_text)
    return processed_text

#### lowercasing

In [ ]:
df_train["drugName"] = df_train["drugName"].apply(lambda x: str(x).lower())
df_test["drugName"] = df_test["drugName"].apply(lambda x: str(x).lower())

df_train["condition"] = df_train["condition"].apply(lambda x: str(x).lower())
df_test["condition"] = df_test["condition"].apply(lambda x: str(x).lower())

In [ ]:
np.sort(df_train["condition"].unique())

array(['abdominal distension', 'abnormal uterine bleeding', 'abortion',
       'acetaminophen overdose', 'acial lipoatrophy', 'acial wrinkles',
       'acne', 'actinic keratosis', 'actor ix deficiency',
       'acute coronary syndrome', 'acute lymphoblastic leukemia',
       'acute nonlymphocytic leukemia', 'acute promyelocytic leukemia',
       "addison's disease", 'adhd', 'adrenocortical insufficiency',
       'adult human growth hormone deficiency', 'aggressive behavi',
       'agitated state', 'agitation', 'aids related wasting',
       'ailure to thrive', 'alcohol dependence', 'alcohol withdrawal',
       'allergic reactions', 'allergic rhinitis', 'allergic urticaria',
       'allergies', 'alopecia', 'alpha-1 proteinase inhibitor deficiency',
       "alzheimer's disease", 'amebiasis', 'amenorrhea',
       'amilial cold autoinflammatory syndrome',
       'amilial mediterranean feve', 'amyotrophic lateral sclerosis',
       'anal fissure and fistula', 'anal itching', 'anaphylaxis',


#### replace garbage with apostrophes

In [ ]:
df_train["review"] = df_train["review"].str.replace("&#039;", "'")

NameError: name 'df_train' is not defined

In [ ]:
# pip install scikit-learn

#### one hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore")

In [ ]:
dn_train_ohe = ohe.fit_transform(df_train["drugName"].values.reshape(-1,1)).toarray()
dn_test_ohe = ohe.transform(df_test["drugName"].values.reshape(-1,1)).toarray()

cond_train_ohe = ohe.fit_transform(df_train["condition"].values.reshape(-1,1)).toarray()
cond_test_ohe = ohe.transform(df_test["condition"].values.reshape(-1,1)).toarray()

In [ ]:
dn_train_ohe.shape, cond_train_ohe.shape

((159498, 3412), (159498, 811))

In [ ]:
dn_test_ohe.shape, cond_test_ohe.shape

((53200, 3412), (53200, 811))

#### lowercasing and placing apostrophes (again)

In [ ]:
#lowercase the train data
df_train['processed_review'] = df_train['review'].apply(lambda x: str(x).lower())

In [ ]:
#lowercase the test data
df_test['processed_review'] = df_test['review'].apply(lambda x: str(x).lower())

In [ ]:
df_train.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,valsartan,left ventricular dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,2012-05-20,"""it has no side effect, i take it in combinati..."
1,guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,2010-04-27,"""my son is halfway through his fourth week of ..."
2,lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,2009-12-14,"""i used to take another oral contraceptive, wh..."
3,ortho evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,2015-11-03,"""this is my first time using any form of birth..."
4,buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,2016-11-27,"""suboxone has completely turned my life around..."


In [ ]:
df_test.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,mirtazapine,depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,2012-02-28,"""i&#039;ve tried a few antidepressants over th..."
1,mesalamine,"crohn's disease, maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,2009-05-17,"""my son has crohn&#039;s disease and has done ..."
2,bactrim,urinary tract infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,2017-09-29,"""quick reduction of symptoms"""
3,contrave,weight loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,2017-03-05,"""contrave combines drugs that were used for al..."
4,cyclafem 1 / 35,birth control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,2015-10-22,"""i have been on this birth control for one cyc..."


In [ ]:
df_train['processed_review'] = df_train['review'].str.replace('&#039;', "'", regex=False)
df_test['processed_review'] = df_test['review'].str.replace('&#039;', "'", regex=False)

In [ ]:
df_test.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,mirtazapine,depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,2012-02-28,"""I've tried a few antidepressants over the yea..."
1,mesalamine,"crohn's disease, maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,2009-05-17,"""My son has Crohn's disease and has done very ..."
2,bactrim,urinary tract infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,2017-09-29,"""Quick reduction of symptoms"""
3,contrave,weight loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,2017-03-05,"""Contrave combines drugs that were used for al..."
4,cyclafem 1 / 35,birth control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,2015-10-22,"""I have been on this birth control for one cyc..."


In [ ]:
preprocess('"Hello World"')

' hello world '

#### applying preprocess(text)

In [ ]:
nltk.download("all")
!pip install nltk
!pip install punkt

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/stevenliu/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/stevenliu/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/stevenliu/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/stevenliu/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/stevenliu/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Down

ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


In [ ]:
df_train["processed_review"] = df_train["processed_review"].apply(preprocess)
df_test["processed_review"] = df_test["processed_review"].apply(preprocess)

In [ ]:
df_train = df_train.drop(columns=["review"])
df_test = df_test.drop(columns=["review"])

KeyError: "['review'] not found in axis"

In [ ]:
df_test.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,processed_review
0,mirtazapine,depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22,2012-02-28,ve tried antidepressant year citalopram fluox...
1,mesalamine,"crohn's disease, maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17,2009-05-17,son crohn s disease done well asacol complain...
2,bactrim,urinary tract infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3,2017-09-29,quick reduction symptom
3,contrave,weight loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35,2017-03-05,contrave combine drug used alcohol smoking op...
4,cyclafem 1 / 35,birth control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4,2015-10-22,birth control one cycle reading review type s...


#### saving processed data to csv

In [ ]:
df_train.to_csv("DrugReviewAnalysis/drugsTrain_processed.csv", sep="\t", index=False)
df_test.to_csv("DrugReviewAnalysis/drugsTest_processed.csv", sep="\t", index=False)

#df_train.to_csv("drugsTrain_processed.csv", sep="\t", index=False)
#df_test.to_csv("drugsTest_processed.csv", sep="\t", index=False)